In [0]:
! pip install pymorphy2


In [0]:
import bz2
from tqdm import tqdm
import json
import pandas as pd
import numpy as np
import nltk
from gensim.models import word2vec
from sklearn.model_selection import train_test_split
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
responses = []
with bz2.BZ2File('/content/drive/My Drive/Colab Notebooks/banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:22, 2427.18it/s]


In [0]:
# Созданим датаферейм из отзывово с 1 и 5 оценками

In [0]:
rating=[]
title=[]
text_ds=[]

In [0]:
for respons in responses:
    if respons['rating_grade']==5:
        text_ds.append(respons['text'])
        rating.append(5)
        title.append(respons['title'])
        
    elif respons['rating_grade']==1:
        text_ds.append(respons['text'])
        rating.append(1)
        title.append(respons['title'])
        

In [0]:
df=pd.DataFrame({'text': text_ds,
                'title': title,
                'rating':rating})

In [7]:
df.head()

,text,title,rating
0,Открыт вклад и счет в USD. Плюс к этому есть з...,Некомпетентность сотрудников банка,1
1,Доброго времени! Вчера мне поступило смс-уведо...,Навязывание страховки,1
2,"05.06.2015г. около 15 часов, пришел в указанны...",Нарушение договора банковского вклада,1
3,Для оплаты коммунальных платежей пользуюсь пла...,"Невозможно оплатить отчисления на капремонт ""Т...",1
4,В апреле этого года пришла в отделение сбербан...,Некомпетентность сотрудника Сбербанка при пров...,1


In [8]:
df.shape

(62100, 3)

In [0]:
# Обработка текста

In [9]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 7.1MB 8.8MB/s 


In [0]:
m = pymorphy2.MorphAnalyzer()


In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def lemmatize(text, pymorphy=m):
    worlds=[]
    for w in text:
      worlds.append(pymorphy.parse(w)[0].normal_form)
    return worlds
    

In [0]:
def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = re.sub("[^а-яА-Я]"," ", review)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words('russian')
        words = [w for w in words if not w in stops]
    return(words)
    

def review_to_sentences(review, remove_stopwords=False):
    raw_sentences = sent_tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(lemmatize(review_to_wordlist(raw_sentence, remove_stopwords)))
    return sentences



In [62]:
len(df.text)

62100

In [65]:
%%time
corpus_sentences = []
i=0
for text in df.text:
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(df.text)))
    i+=1
    if type(text) == float:
        continue
    sentences = review_to_sentences(text)
    corpus_sentences += sentences

Processed 0 of 62100
Processed 1000 of 62100
Processed 2000 of 62100
Processed 3000 of 62100
Processed 4000 of 62100
Processed 5000 of 62100
Processed 6000 of 62100
Processed 7000 of 62100
Processed 8000 of 62100
Processed 9000 of 62100
Processed 10000 of 62100
Processed 11000 of 62100
Processed 12000 of 62100
Processed 13000 of 62100
Processed 14000 of 62100
Processed 15000 of 62100
Processed 16000 of 62100
Processed 17000 of 62100
Processed 18000 of 62100
Processed 19000 of 62100
Processed 20000 of 62100
Processed 21000 of 62100
Processed 22000 of 62100
Processed 23000 of 62100
Processed 24000 of 62100
Processed 25000 of 62100
Processed 26000 of 62100
Processed 27000 of 62100
Processed 28000 of 62100
Processed 29000 of 62100
Processed 30000 of 62100
Processed 31000 of 62100
Processed 32000 of 62100
Processed 33000 of 62100
Processed 34000 of 62100
Processed 35000 of 62100
Processed 36000 of 62100
Processed 37000 of 62100
Processed 38000 of 62100
Processed 39000 of 62100
Processed 400

In [79]:
corpus_sentences[10]

['далее',
 'после',
 'долгий',
 'около',
 'два',
 'часы',
 'выяснение',
 'отношение',
 'с',
 'операционист',
 'и',
 'видимо',
 'она',
 'начальник',
 'они',
 'попытка',
 'отменить',
 'один',
 'транзакция',
 'и',
 'том',
 'далее']

In [0]:
# status = open('/content/drive/My Drive/Colab Notebooks/corpus_sentences.txt','w', encoding='utf-8')
# status.write(corpus_sentences)
# status.close()
with open('/content/drive/My Drive/Colab Notebooks/corpus_sentences.txt', 'w', encoding='utf-8') as f:
    for item in corpus_sentences:
        f.write("%s\n" % item)

In [67]:
len(corpus_sentences)

1197275

In [0]:
#Обучаем и сохроняем модель

In [0]:
model_df = word2vec.Word2Vec(corpus_sentences, workers=6, size=300, min_count=3, window=5, sample=1e-3)

In [69]:
model_df.save('/content/drive/My Drive/Colab Notebooks/model_df')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [70]:
print(len(model_df.wv.vocab))

31239


In [0]:
# тест на аналогии

In [172]:
print(model_df.wv.most_similar(positive=["счёт", "клиент"], negative=["время"], topn=1))

[('вкладчик', 0.4370534121990204)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# тест на определение ближайших слов

In [78]:
print(model_df.wv.most_similar("счёт", topn=10))

[('картсчёт', 0.7556276917457581), ('сч', 0.7494080066680908), ('скс', 0.6314352750778198), ('сберкнижка', 0.6093786358833313), ('депозит', 0.5608007907867432), ('карта', 0.5233497619628906), ('кошелёк', 0.5000457763671875), ('карточка', 0.49727413058280945), ('рублёвый', 0.4960053563117981), ('вклад', 0.4945300817489624)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# тест на определение лишнего слова

In [84]:
print(model_df.wv.doesnt_match("начальник карта банк".split()))

начальник


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# TSNE для наглядного предславления на плоскости

In [0]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []



In [88]:
fd = FreqDist()
for s in tqdm(corpus_sentences):
    fd.update(s)

for w in fd.most_common(200):
    top_words.append(w[0])
    
print(top_words[:40:])
top_words_vec = model_df[top_words]


['в', 'я', 'и', 'не', 'на', 'что', 'с', 'быть', 'банка', 'по', 'это', 'карта', 'банк', 'весь', 'а', 'мой', 'как', 'за', 'к', 'но', 'то', 'о', 'деньга', 'у', 'день', 'год', 'отделение', 'тот', 'который', 'кредит', 'так', 'сотрудник', 'они', 'вы', 'клиент', 'она', 'мочь', 'счёт', 'он', 'сказать', 'же', 'один', 'от', 'свой', 'для', 'через', 'уже', 'этот', 'заявление', 'из', 'такой', 'до', 'сумма', 'мы', 'раз', 'вопрос', 'время', 'рубль', 'том', 'ещё']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [102]:
top_words_200=model_df[top_words][:200:]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [115]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_200)

CPU times: user 954 ms, sys: 996 µs, total: 955 ms
Wall time: 957 ms


In [116]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (модель, топ 200 слов)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

In [0]:
# Осмысленные кластеры слов(время:минута неделя месяц, обращение: заявка обращение претензия заявление)

In [117]:
print(model_df.wv.most_similar("хорошо", topn=10))

[('отлично', 0.6508923768997192), ('здорово', 0.6294060945510864), ('неплохо', 0.5911627411842346), ('ок', 0.5875182747840881), ('плохо', 0.5615386962890625), ('идеально', 0.552628219127655), ('классно', 0.5479950904846191), ('гладко', 0.5237932205200195), ('красиво', 0.5157138109207153), ('гуд', 0.5083898305892944)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# Координаты для нового простанства

In [0]:
x_aix=model_df.wv['хорошо']-model_df.wv['плохо']

In [0]:
y_aix=model_df.wv['быстро']-model_df.wv['медленно']

In [0]:
# Найдем популярные банки

In [0]:
import collections

c = collections.Counter()

In [0]:
c = collections.Counter()
for banky in responses:
    c[banky['bank_name']]+=1

In [0]:
banks=c.most_common(10)

In [139]:
for bank in banks:
  print(bank[0].lower())

сбербанк россии
альфа-банк
втб 24
русский стандарт
хоум кредит банк
тинькофф банк
национальный банк «траст»
ренессанс кредит
связной банк
отп банк


In [0]:
banks=['сбербанк', 'альфа','тинькофф','траст', 'ренессанс','связной','отп']

In [0]:
X=[] #хорошо плохо
Y=[] #быстро медленно
for bank in banks:
  z=model_df.wv[bank]
  X.append(x_aix*z)
  Y.append(y_aix*z)


In [0]:
# Снизим размерность

In [0]:
tsne = TSNE(n_components=1, random_state=0)
top_words_tsne_x = tsne.fit_transform(X)
top_words_tsne_y = tsne.fit_transform(Y)

In [0]:
# Построим зависимостри на двухмерной плоскости

In [165]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Ось Х 'хорошо-плохо', Ось Y 'быстро-медленно'")

source = ColumnDataSource(data=dict(x1=top_words_tsne_x,
                                    x2=top_words_tsne_y,
                                    names=banks))

p.scatter(x="x1", y="x2", size=20, source=source)
p.axis

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="10pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)